In [1]:
import gym
import ray
from ray.rllib.agents.ppo import PPOTrainer
from scouting_gym.scouting_env_real import ScoutingEnvInference
from gym.envs.registration import register
from IPython.display import clear_output

register(
    id='Scouting-inference-v0',
    entry_point='scouting_gym.scouting_env_real:ScoutingEnvInference'
)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
env = gym.make('Scouting-inference-v0')

print(env.observation_space)

[ERROR] [1611844845.193488]: Current /scan not ready yet, retrying for getting laser_scan


(array([0.7343741 , 2.4961479 , 2.3182755 , 3.1681209 , 2.053265  ,
       2.3256779 , 3.837255  , 3.0036259 , 1.0221846 , 2.302958  ,
       2.1137152 , 3.672243  , 2.9076405 , 2.3092177 , 0.44011754,
       2.4117916 , 2.531386  , 0.30688444], dtype=float32), array([8.151919 , 1.4234198], dtype=float32), array([0.4507907 , 0.84739566], dtype=float32))
Tuple(Box(0.0, 4.0, (18,), float32), Box(-10.0, 10.0, (2,), float32), Box(-1.0, 1.0, (2,), float32))


In [3]:
obs = env.reset()
obs

[-3.3814182031314273, -1.8544499186115495, -0.11191840813799239]


(array([1.7717302 , 3.4954677 , 2.2796528 , 1.7497336 , 1.3118002 ,
        0.69589406, 0.7038929 , 0.7638837 , 1.293803  , 3.7794247 ,
        1.5857586 , 1.1878191 , 1.3277978 , 1.2798052 , 1.4877734 ,
        3.6354465 , 1.9277065 , 1.7157388 ], dtype=float32),
 array([0.0814182 , 0.15444992], dtype=float32),
 array([-0.99, -0.99], dtype=float32))

In [4]:
print(env.offs_x, env.offs_y)

-1.9829870348639111 -1.744520828712645


In [12]:
import time
env.reset()
for _ in range(10):
    #obs, reward, done, _ = env.step(1)
    time.sleep(0.5)
    obs, reward, done, _ = env.step([0.0, ])

    clear_output(wait=True)
    print(obs[1], obs[2])
    if done:
        print('break')
        break
print(obs)
env.reset()

[0.50019634 0.2855621 ] [ 0.03515336 -0.01265672]
break
(array([3.145521  , 3.3794854 , 0.7918794 , 0.74988586, 0.63790286,
       0.6199056 , 0.62790436, 0.6938943 , 1.0258437 , 2.5716083 ,
       3.331493  , 3.1875148 , 2.6975894 , 0.78588027, 0.8258742 ,
       0.81787544, 1.4857738 , 3.4374766 ], dtype=float32), array([0.50019634, 0.2855621 ], dtype=float32), array([ 0.03515336, -0.01265672], dtype=float32))
[-1.8846205486223986, -1.684338355202867, -0.11227330576711596]


(array([3.209511  , 3.4034817 , 0.7778815 , 0.76788306, 0.6299041 ,
        0.6019084 , 0.6119068 , 0.68189615, 0.99184895, 2.5456123 ,
        3.3054967 , 3.1715171 , 2.7115872 , 0.7978785 , 0.79387903,
        0.84187186, 1.4977719 , 3.4754708 ], dtype=float32),
 array([0.48462054, 0.28433836], dtype=float32),
 array([-0.99, -0.99], dtype=float32))

In [5]:
print(obs)

(array([2.371639 , 1.5197686, 1.083835 , 1.5697609, 3.99     , 1.7457341,
       0.9118612, 0.781881 , 0.6758971, 0.6519007, 1.18182  , 1.333797 ,
       1.549764 , 3.1415217, 2.2296605, 2.3456428, 2.371639 , 2.547612 ],
      dtype=float32), array([0.0542413 , 0.32386878], dtype=float32), array([0.99     , 0.1522624], dtype=float32))


In [4]:
ray.init()

2021-01-28 15:40:51,592	INFO resource_spec.py:223 -- Starting Ray with 6.54 GiB memory available for workers and up to 3.29 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2021-01-28 15:40:52,362	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.178.21',
 'raylet_ip_address': '192.168.178.21',
 'redis_address': '192.168.178.21:6379',
 'object_store_address': '/tmp/ray/session_2021-01-28_15-40-51_587588_48670/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-01-28_15-40-51_587588_48670/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2021-01-28_15-40-51_587588_48670'}

In [5]:
config = {
    "env": ScoutingEnvInference,  # or "corridor" if registered above
    "env_config": {
        "corridor_length": 5,
    },
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.

    "num_gpus": 0,
    "num_workers": 1,  # parallelism
    "model": {
        "fcnet_hiddens": [256, ],
        #"fcnet_hiddens": tune.grid_search([[64, 64, ], [128, 128, ], [256, 256, ]])
    }
}

In [6]:
def load(checkpoint_path, config):
    """
    Load a trained RLlib agent from the specified path. Call this before testing a trained agent.
    :param path: Path pointing to the agent's saved checkpoint (only used for RLlib agents)
    """
    agent = PPOTrainer(config=config)
    agent.restore(checkpoint_path)
    return agent
import numpy as np
from gym import spaces
def test(agent, env):
    """Test trained agent for a single episode. Return the episode reward"""
    # instantiate env class

    # run until episode ends
    episode_reward = 0
    done = False
    obs = env.reset()
    while not done:
        action = agent.compute_action(obs)
        obs, reward, done, info = env.step(action)
        clear_output(wait=True)
        print(obs[1])
        episode_reward += reward
    env.reset()
    return episode_reward

In [7]:
checkpoint_path = '/home/dschori/Downloads/checkpoint_51/checkpoint-51'

In [8]:
agent = load(checkpoint_path=checkpoint_path, config=config)

2021-01-28 15:40:59,001	INFO trainer.py:605 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-01-28 15:40:59,005	INFO trainer.py:630 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=48785) WARNING:tensorflow:From /home/dschori/.local/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=48785) Instructions for updating:
(pid=48785) non-resource variables are not supported in the long term
2021-01-28 15:41:08,102	WARNING util.py:37 -- Install gputil for GPU system monitoring.
2021-01-28 15:41:08,221	INFO trainable.py:472 -- Restored on 192.168.178.21 from checkpoint: /home/dschori/Downloads/checkpoint_51/checkpoint-51
2021-01-28 15:41:08,222	INFO trainable.py:480 -- Current state after restoring: {'_iteration': 51, '_timesteps_total': 

(array([2.4448283 , 3.0445101 , 2.6791432 , 3.740952  , 0.20781107,
       3.4154186 , 2.164655  , 3.6742406 , 0.52097183, 0.13686216,
       2.1084461 , 1.2485379 , 1.0531713 , 2.70111   , 1.5785227 ,
       1.6416779 , 2.668904  , 3.1490288 ], dtype=float32), array([ 2.0586426, -6.165655 ], dtype=float32), array([0.50621814, 0.9000319 ], dtype=float32))


In [24]:
env.target_p = (-2., -1.6)

In [25]:
episode_reward = test(agent=agent, env=env)
episode_reward

[-0.47864813  0.04292046]
[-1.5550143026821146, -1.6592586383164387, -0.11195966085212106]


116.61992595477511

In [20]:
agent.compute_action(env.reset())

[-4.76677755406737, -2.1496847361693803, -0.10936877266553566]


array([0.29416484], dtype=float32)